## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
0,0,Busselton,AU,-33.6500,115.3333,65.08,89,0,15.55,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,49.66,87,100,17.27,light rain
2,2,Manta,EC,-0.9500,-80.7333,82.18,74,40,8.05,scattered clouds
3,3,Sabang,ID,5.8933,95.3214,82.29,71,19,9.73,few clouds
4,4,Bubaque,GW,11.2833,-15.8333,78.19,72,100,21.52,overcast clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
2,2,Manta,EC,-0.9500,-80.7333,82.18,74,40,8.05,scattered clouds
3,3,Sabang,ID,5.8933,95.3214,82.29,71,19,9.73,few clouds
4,4,Bubaque,GW,11.2833,-15.8333,78.19,72,100,21.52,overcast clouds
7,7,San Borja,BO,-14.8167,-66.8500,81.39,74,76,3.29,broken clouds
10,10,Rikitea,PF,-23.1203,-134.9692,77.29,74,100,12.80,light rain
21,21,Atuona,PF,-9.8000,-139.0333,77.23,80,29,14.54,light rain
23,23,Ixtapa,MX,20.7000,-105.2000,86.63,34,0,7.09,clear sky
27,27,Vaini,TO,-21.2000,-175.2000,81.34,79,100,17.36,light rain
30,30,Bambanglipuro,ID,-7.9500,110.2833,78.04,85,100,4.68,overcast clouds
31,31,Touros,BR,-5.1989,-35.4608,83.30,70,100,10.67,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID         193
City            193
Country         191
Lat             193
Lng             193
Max Temp        193
Humidity        193
Cloudiness      193
Wind Speed      193
Weather Desc    193
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [18]:
clean_df.count()

City_ID         191
City            191
Country         191
Lat             191
Lng             191
Max Temp        191
Humidity        191
Cloudiness      191
Wind Speed      191
Weather Desc    191
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Desc", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Desc,Lat,Lng,Hotel Name
2,Manta,EC,82.18,scattered clouds,-0.9500,-80.7333,
3,Sabang,ID,82.29,few clouds,5.8933,95.3214,
4,Bubaque,GW,78.19,overcast clouds,11.2833,-15.8333,
7,San Borja,BO,81.39,broken clouds,-14.8167,-66.8500,
10,Rikitea,PF,77.29,light rain,-23.1203,-134.9692,
21,Atuona,PF,77.23,light rain,-9.8000,-139.0333,
23,Ixtapa,MX,86.63,clear sky,20.7000,-105.2000,
27,Vaini,TO,81.34,light rain,-21.2000,-175.2000,
30,Bambanglipuro,ID,78.04,overcast clouds,-7.9500,110.2833,
31,Touros,BR,83.30,overcast clouds,-5.1989,-35.4608,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Desc} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))